In [6]:
import pandas as pd
from mp_api.client import MPRester

from matminer.featurizers.conversions import StrToComposition
from matminer.featurizers.composition import ElementProperty
from matminer.featurizers.conversions import CompositionToOxidComposition
from matminer.featurizers.composition import OxidationStates
from matminer.featurizers.structure import DensityFeatures
from matminer.featurizers.composition.orbital import ValenceOrbital
from matminer.featurizers.composition.element import Stoichiometry
from matminer.featurizers.composition.ion import IonProperty

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.model_selection import KFold, cross_val_score
from figrecipes import PlotlyFig
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import shap
from flatten_dict import flatten

In [7]:
with MPRester("4aBu5iIVl9Dt8fm9LGXMKkblVrhBpk10") as mpr:
    docs_di = mpr.materials.dielectric.search()

flattened = [{
    k: v for k, v in flatten(doc.dict(), reducer="dot").items() if k != "fields_not_requested"
} for doc in docs_di]

df = pd.DataFrame.from_records(flattened)
df

Retrieving DielectricDoc documents:   0%|          | 0/7287 [00:00<?, ?it/s]

,builder_meta.emmet_version,builder_meta.pymatgen_version,builder_meta.pull_request,builder_meta.database_version,builder_meta.build_date,builder_meta.license,nsites,elements,nelements,composition.Mg,...,composition.U,composition_reduced.U,composition.Gd,composition_reduced.Gd,composition.Ce,composition_reduced.Ce,composition.Eu,composition_reduced.Eu,composition.Xe,composition_reduced.Xe
0,0.63.1,2023.7.17,990,2023.11.1,2023-08-09 06:53:02.264000,None,8,"[Mg, N, Zr]",3,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.63.1,2023.7.17,990,2023.11.1,2023-08-09 06:53:03.092000,None,28,"[As, O]",2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.63.1,2023.7.17,990,2023.11.1,2023-08-09 06:53:03.142000,None,14,"[Cs, Se, Si]",3,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.63.1,2023.7.17,990,2023.11.1,2023-08-09 06:53:04.153000,None,22,"[B, Ca, O]",3,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.63.1,2023.7.17,990,2023.11.1,2023-08-09 06:53:02.432000,None,12,"[Cs, O, Sc, W]",4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7282,0.63.1,2023.7.17,990,2023.11.1,2023-08-09 06:53:18.556000,None,9,"[Cs, I, Pd]",3,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7283,0.63.1,2023.7.17,990,2023.11.1,2023-08-09 06:53:15.866000,None,11,"[C, Ca, K, O]",4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7284,0.63.1,2023.7.17,990,2023.11.1,2023-08-09 06:53:16.463000,None,20,"[P, Se, Sn]",3,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7285,0.63.1,2023.7.17,990,2023.11.1,2023-08-09 06:53:17.333000,None,20,"[F, Nb, O]",3,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
df = df.dropna(axis = 1)
df.columns.values

array(['builder_meta.emmet_version', 'builder_meta.pymatgen_version',
       'builder_meta.pull_request', 'builder_meta.database_version',
       'builder_meta.build_date', 'nsites', 'elements', 'nelements',
       'formula_pretty', 'formula_anonymous', 'chemsys', 'volume',
       'density', 'density_atomic', 'symmetry.crystal_system',
       'symmetry.symbol', 'symmetry.number', 'symmetry.point_group',
       'symmetry.symprec', 'symmetry.version', 'property_name',
       'material_id', 'deprecated', 'last_updated', 'origins', 'warnings',
       'total', 'ionic', 'electronic', 'e_total', 'e_ionic',
       'e_electronic', 'n'], dtype=object)

In [ ]:
from emmet.core.summary import HasProps

with MPRester("4aBu5iIVl9Dt8fm9LGXMKkblVrhBpk10") as mpr:
    docs1 = mpr.materials.summary.search(has_props = [HasProps.dielectric])
flattened1 = [{
    k: v for k, v in flatten(doc.dict(), reducer="dot").items() if k != "fields_not_requested"
} for doc in docs1]
df1 = pd.DataFrame.from_records(flattened1)
df1.columns.values

In [ ]:
import shelve

filename='shelve.out'
my_shelf = shelve.open(filename,'n') # 'n' for new

for key in dir():
    try:
        my_shelf[key] = globals()[key]
    except:
        print('ERROR shelving: {0}'.format(key))
my_shelf.close()

In [ ]:
import shelve
filename='shelve.out'
my_shelf = shelve.open(filename)
my_shelf

In [ ]:
varlist = globals()
for key in my_shelf:
    if key not in varlist and key[0] != '_' and key[0] < 'e':
        globals()[key] = my_shelf[key]

In [ ]:
varlist = globals()
for key in my_shelf:
    if key not in varlist and key[0] != '_' and key[0] >= 'e' and key[0] < 'm':
        globals()[key] = my_shelf[key]

In [ ]:
varlist = globals()
for key in my_shelf:
    if key not in varlist and key[0] != '_' and key[0] >= 'n':
        globals()[key] = my_shelf[key]